<a href="https://colab.research.google.com/github/sangeetsaurabh/tweet_sentiment_extraction/blob/master/embedding_bag_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This Notebook is being created to train and embedding by predicting the emotion of the tweet. Goal to train the embedding it to visualize positive, negative and neutral words in Training and Test dataset.

In [1]:
!pip3 install torchtext==0.6.0

     |████████████████████████████████| 71kB 2.0MB/s 
     |████████████████████████████████| 1.1MB 6.9MB/s 
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1


#### Mount the Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
data_folder = "/content/drive/My Drive/tweet-sentiment-extraction/data"
tmp_folder = '/tmp'

In [4]:
import sys,os
sys.path.append("/content/drive/My Drive/")
os.getcwd()

'/content'

In [5]:
import util
from util.translation import *

In [6]:
import pandas as pd
import re
from collections import Collection

import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import TranslationDataset
from torchtext.data import Field, BucketIterator,Pipeline,LabelField
from torchtext.data.metrics import bleu_score

import spacy
import numpy as np

import random
import math
import time
import matplotlib.pyplot as plt


import warnings
warnings.filterwarnings('ignore')

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

#### Load Training and Test dataset

In [8]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)

train_data = pd.read_csv(data_folder + "/train.csv")
test_data = pd.read_csv(data_folder + "/test.csv")

print(train_data.shape)
print(test_data.shape)

(27481, 4)
(3534, 3)


In [9]:
### Removing rows with null value
train_data = train_data[train_data.text == train_data.text]
test_data = test_data[test_data.text == test_data.text]

In [10]:
### Drop the neutral for now
train_data = train_data[train_data.sentiment != 'neutral']
test_data = test_data[test_data.sentiment != 'neutral']

print(train_data.shape)
print(test_data.shape)

(16363, 4)
(2104, 3)


In [11]:
test_data.head()

,textID,text,sentiment
1,96d74cb729,Shanghai is also really exciting (precisely -- skyscrapers galore). Good tweeps in China: (SH) (BJ).,positive
2,eee518ae67,"Recession hit Veronique Branquinho, she has to quit her company, such a shame!",negative
3,01082688c6,happy bday!,positive
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive
5,726e501993,that`s great!! weee!! visitors!,positive


In [12]:
train_data.head(5)

,textID,text,selected_text,sentiment
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on the releases we already bought","Sons of ****,",negative
6,6e0c6d75b1,2am feedings for the baby are fun when he is all smiles and coos,fun,positive


In [14]:
train_data.sentiment.value_counts()

positive    8582
negative    7781
Name: sentiment, dtype: int64

In [15]:
test_data.sentiment.value_counts()

positive    1103
negative    1001
Name: sentiment, dtype: int64

#### Save the data to load into Torch dataset

In [16]:
train_data["idx"] = train_data.index
test_data["idx"] = test_data.index

In [17]:
train_data["sentiment_label"] = np.where(train_data.sentiment == 'positive', 1, 0)
test_data["sentiment_label"] = np.where(test_data.sentiment == 'positive', 1, 0)

In [18]:
train_data.head()

,textID,text,selected_text,sentiment,idx,sentiment_label
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,1,0
2,088c60f138,my boss is bullying me...,bullying me,negative,2,0
3,9642c003ef,what interview! leave me alone,leave me alone,negative,3,0
4,358bd9e861,"Sons of ****, why couldn`t they put them on the releases we already bought","Sons of ****,",negative,4,0
6,6e0c6d75b1,2am feedings for the baby are fun when he is all smiles and coos,fun,positive,6,1


In [19]:
test_data.tail()

,textID,text,sentiment,idx,sentiment_label
3529,e5f0e6ef4b,"its at 3 am, im very tired but i can`t sleep but i try it",negative,3529,0
3530,416863ce47,"All alone in this old house again. Thanks for the net which keeps me alive and kicking! Whoever invented the net, i wanna kiss your hair!",positive,3530,1
3531,6332da480c,I know what you mean. My little dog is sinking into depression... he wants to move someplace tropical,negative,3531,0
3532,df1baec676,_sutra what is your next youtube video gonna be about? I love your videos!,positive,3532,1
3533,469e15c5a8,http://twitpic.com/4woj2 - omgssh ang cute ng bby.!,positive,3533,1


Save the data in a format such that PyTorch out of box data loader can be used to load the data

In [22]:
train_data["text"].to_csv(data_folder + "/train_data.text",index=False,header=False)
train_data["selected_text"].to_csv(data_folder + "/train_data.stext",index=False,header=False)
train_data["sentiment_label"].to_csv(data_folder + "/train_data.sentiment",index=False,header=False)
train_data["idx"].to_csv(data_folder + "/train_data.textid",index=False,header=False)

In [21]:
test_data["text"].to_csv(data_folder + "/test_data.text",index=False,header=False)
test_data["text"].to_csv(data_folder + "/test_data.stext",index=False,header=False)
test_data["sentiment_label"].to_csv(data_folder + "/test_data.sentiment",index=False,header=False)
test_data["idx"].to_csv(data_folder + "/test_data.textid",index=False,header=False)

Save the positive and negative sentiment separately

In [23]:
test_data[test_data.sentiment_label == 1]["text"].to_csv(data_folder + "/test_data_positive.text",index=False,header=False)
test_data[test_data.sentiment_label == 1]["text"].to_csv(data_folder + "/test_data_positive.stext",index=False,header=False)
test_data[test_data.sentiment_label == 1]["sentiment_label"].to_csv(data_folder + "/test_data_positive.sentiment",index=False,header=False)
test_data[test_data.sentiment_label == 1]["idx"].to_csv(data_folder + "/test_data_positive.textid",index=False,header=False)

In [24]:
test_data[test_data.sentiment_label == 0]["text"].to_csv(data_folder + "/test_data_negative.text",index=False,header=False)
test_data[test_data.sentiment_label == 0]["text"].to_csv(data_folder + "/test_data_negative.stext",index=False,header=False)
test_data[test_data.sentiment_label == 0]["sentiment_label"].to_csv(data_folder + "/test_data_negative.sentiment",index=False,header=False)
test_data[test_data.sentiment_label == 0]["idx"].to_csv(data_folder + "/test_data_negative.textid",index=False,header=False)

In [25]:
train_data[train_data.sentiment_label == 1]["text"].to_csv(data_folder + "/train_data_positive.text",index=False,header=False)
train_data[train_data.sentiment_label == 1]["selected_text"].to_csv(data_folder + "/train_data_positive.stext",index=False,header=False)
train_data[train_data.sentiment_label == 1]["sentiment_label"].to_csv(data_folder + "/train_data_positive.sentiment",index=False,header=False)
train_data[train_data.sentiment_label == 1]["idx"].to_csv(data_folder + "/train_data_positive.textid",index=False,header=False)

In [26]:
train_data[train_data.sentiment_label == 0]["text"].to_csv(data_folder + "/train_data_negative.text",index=False,header=False)
train_data[train_data.sentiment_label == 0]["selected_text"].to_csv(data_folder + "/train_data_negative.stext",index=False,header=False)
train_data[train_data.sentiment_label == 0]["sentiment_label"].to_csv(data_folder + "/train_data_negative.sentiment",index=False,header=False)
train_data[train_data.sentiment_label == 0]["idx"].to_csv(data_folder + "/train_data_negative.textid",index=False,header=False)

In [27]:
train_data[(train_data.selected_text.str.contains('disappoint', case=False, regex=False)) & (train_data.sentiment_label == 1)]

,textID,text,selected_text,sentiment,idx,sentiment_label
22638,add1c95bee,Star Trek.. Did not disappoint! 5 star!!,Did not disappoint! 5 star!!,positive,22638,1
23747,73714f7a1a,aww i hope it does fly by because JT episodes are usually really good (and it`s early but so far this ep hassn`t disappointed),aww i hope it does fly by because JT episodes are usually really good (and it`s early but so far this ep hassn`t disappointed),positive,23747,1


#### Load the Torch dataset

In [28]:
from typing import Collection
import html
BOS,EOS,FLD,UNK,PAD = 'xxbos','xxeos','xxfld','xxunk','xxpad'
TK_MAJ,TK_UP,TK_REP,TK_WREP = 'xxmaj','xxup','xxrep','xxwrep'

def spec_add_spaces(t:str) -> str:
    "Add spaces around / and # in `t`. \n"
    #print(t)
    return re.sub(r'([/#\n/.])', r' \1 ', t)


def rm_useless_spaces(t:str) -> str:
    "Remove multiple spaces in `t`."
    return re.sub(' {2,}', ' ', t)


def replace_rep(t:str) -> str:
    "Replace repetitions at the character level in `t`."
    def _replace_rep(m:Collection[str]) -> str:
        c,cc = m.groups()
        return f' {TK_REP} {len(cc)+1} {c} '
    re_rep = re.compile(r'(\S)(\1{3,})')
    return re_rep.sub(_replace_rep, t)

def replace_wrep(t:str) -> str:
    "Replace word repetitions in `t`."
    def _replace_wrep(m:Collection[str]) -> str:
        c,cc = m.groups()
        return f' {TK_WREP} {len(cc.split())+1} {c} '
    re_wrep = re.compile(r'(\b\w+\W+)(\1{3,})')
    return re_wrep.sub(_replace_wrep, t)

def fix_html(x:str) -> str:
    "List of replacements from html strings in `x`."
    re1 = re.compile(r'  +')
    x = x.replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '"').replace('<unk>',UNK).replace(' @.@ ','.').replace(
        ' @-@ ','-').replace(' @,@ ',',').replace('\\', ' \\ ')
    return re1.sub(' ', html.unescape(x))

def replace_all_caps(x:Collection[str]) -> Collection[str]:
    "Replace tokens in ALL CAPS in `x` by their lower version and add `TK_UP` before."
    res = []
    for t in x:
        if t.isupper() and len(t) > 1: res.append(TK_UP); res.append(t.lower())
        else: res.append(t)
    return res

def deal_caps(x:Collection[str]) -> Collection[str]:
    "Replace all Capitalized tokens in `x` by their lower version and add `TK_MAJ` before."
    res = []
    for t in x:
        if t == '': continue
        if t[0].isupper() and len(t) > 1 and t[1:].islower(): res.append(TK_MAJ)
        res.append(t.lower())
    return res

def convert_int(x:Collection[str]) -> Collection[str]:
    "Replace all Capitalized tokens in `x` by their lower version and add `TK_MAJ` before."
    res = []
    for t in x:
        res.append(int(t))
    return res

text_pre_rules = [fix_html, replace_rep, replace_wrep, spec_add_spaces, rm_useless_spaces]
text_post_rules = [replace_all_caps, deal_caps]
numeric_rules = [convert_int]

In [29]:
def sequential_transforms(transforms=[]):
    def _forward(txt_input):
        #print(txt_input)
        for transform in transforms:
            #print(transform)
            txt_input = transform(txt_input)
        return txt_input
    return _forward

In [30]:
text_preprocess = sequential_transforms(text_pre_rules)

text_post_preprocess = sequential_transforms(text_post_rules)

numeric_post_preprocess = sequential_transforms(numeric_rules)

In [31]:
spacy_en = spacy.load('en')

In [32]:
def tokenize_src(text):
    """
    Tokenizes text from a string into a list of strings (tokens) and reverses it
    """
    #return [tok.text for tok in spacy_en.tokenizer(text)][::-1]
    text = text_preprocess(text)
    return [tok.text for tok in spacy_en.tokenizer(text)]

def tokenize_trg(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    text = text_preprocess(text)
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [33]:
src_sentences = Field(tokenize = tokenize_src, 
              init_token = '<sos>', 
              eos_token = '<eos>', 
              lower = False,
              preprocessing = text_post_preprocess,
              include_lengths = True)
              #postprocessing = text_post_preprocess)

trg_sentences = Field(tokenize = tokenize_trg,
              init_token = '<sos>', 
              eos_token = '<eos>', 
              lower = False,
              preprocessing = text_post_preprocess,
              include_lengths = True)
              #postprocessing = text_post_preprocess)

#label_sentences = Field(sequential=False,use_vocab=False,postprocessing=numeric_post_preprocess)
label_sentences = LabelField(dtype = torch.float, use_vocab=False, preprocessing=float)
              #init_token = '<sos>', 
              #eos_token = '<eos>', 
              #lower = False,
              #preprocessing = text_post_preprocess,
              #include_lengths = True)
              #postprocessing = text_post_preprocess)

idx_sentences = Field(sequential=False,use_vocab=False,preprocessing=int) #,unk_token=None)

Build the Train and Test Datasets

In [34]:
(train_dataset, test_dataset) = TranslationDataset_c.splits(exts = ('.text','.sentiment','.textid','.stext'), path=data_folder,
                                                    fields = (src_sentences,label_sentences,idx_sentences, trg_sentences),train='train_data',validation='test_data',test=None)

In [35]:
vars(train_dataset.examples[1])

{'idx': 2,
 'label': 0.0,
 'src': ['my', 'boss', 'is', 'bullying', 'me', '.', '.', '.'],
 'trg': ['bullying', 'me']}

In [36]:
vars(test_dataset.examples[1])

{'idx': 2,
 'label': 0.0,
 'src': ['"',
  'xxmaj',
  'recession',
  'hit',
  'xxmaj',
  'veronique',
  'xxmaj',
  'branquinho',
  ',',
  'she',
  'has',
  'to',
  'quit',
  'her',
  'company',
  ',',
  'such',
  'a',
  'shame',
  '!',
  '"'],
 'trg': ['"',
  'xxmaj',
  'recession',
  'hit',
  'xxmaj',
  'veronique',
  'xxmaj',
  'branquinho',
  ',',
  'she',
  'has',
  'to',
  'quit',
  'her',
  'company',
  ',',
  'such',
  'a',
  'shame',
  '!',
  '"']}

In [37]:
print(vars(train_dataset.examples[20]))

{'src': ['i', 'hope', 'unni', 'will', 'make', 'the', 'audition', '.', 'fighting', 'dahye', 'unni', '!'], 'label': 1.0, 'idx': 31, 'trg': ['hope']}


In [38]:
print(vars(test_dataset.examples[19]))

{'src': ['look', 'who', 'i', 'found', 'just', 'for', 'you', '---', '>', 'http', ':', '/', '/', 'twitter', '.', 'com', '/', 'xxup', 'djt2009'], 'label': 1.0, 'idx': 28, 'trg': ['look', 'who', 'i', 'found', 'just', 'for', 'you', '---', '>', 'http', ':', '/', '/', 'twitter', '.', 'com', '/', 'xxup', 'djt2009']}


In [39]:
test_data[test_data.idx == 28]

,textID,text,sentiment,idx,sentiment_label
28,1fa8e6ad66,look who I found just for you ---> http://twitter.com/DJT2009,positive,28,1


In [40]:
print(f"Number of training examples: {len(train_dataset.examples)}")
print(f"Number of validation examples: {len(test_dataset.examples)}")

Number of training examples: 16363
Number of validation examples: 2104


Build the vocab

In [41]:
src_sentences.build_vocab(train_dataset, min_freq = 1)
#label_sentences.build_vocab(train_dataset,min_freq = 1)
trg_sentences.build_vocab(train_dataset, min_freq = 1)
#idx_sentences.build_vocab(train_dataset, min_freq = 1)

print(f"Unique tokens in source (Text) vocabulary: {len(src_sentences.vocab)}")
print(f"Unique tokens in target (Special Text) vocabulary: {len(trg_sentences.vocab)}")

Unique tokens in source (Text) vocabulary: 18956
Unique tokens in target (Special Text) vocabulary: 7069


#### Build the Embedding Bag model

In [42]:
import torch.nn as nn
import torch.nn.functional as F
class TextSentiment(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [43]:
VOCAB_SIZE = len(src_sentences.vocab)
EMBED_DIM = 32
NUM_CLASS = 2
model = TextSentiment(VOCAB_SIZE, EMBED_DIM, NUM_CLASS).to(device)
model

TextSentiment(
  (embedding): EmbeddingBag(18956, 32, mode=mean)
  (fc): Linear(in_features=32, out_features=2, bias=True)
)

#### Train the model

In [44]:
BATCH_SIZE = 32
train_iterator, valid_iterator = BucketIterator.splits(
    (train_dataset, test_dataset), 
    batch_size = BATCH_SIZE, 
    device = device)

In [45]:
def generate_batch(batch):
    #### Build text data
    text = []
    for bs in range(batch.src[0].shape[1]):
      text.append(batch.src[0][:batch.src[1][bs],bs])
    text = torch.cat(text)
    #print(text.shape)

    #### Build offsets
    offset = list(batch.src[1].cpu().detach().numpy())
    offsets = [0] + offset[:-1]
    offsets = torch.tensor(offsets).cumsum(dim=0)
    #print(offsets.shape)
    #print(offsets[-1].item() + offset[-1])

    ### Build label
    label = batch.label.type(torch.LongTensor)
    #print(label.shape)

    ### Build IDx
    idx = batch.idx
    #print(idx.shape)
    
    return text, offsets, label,idx

In [46]:
for i, batch in enumerate(valid_iterator):
  (text, offsets, cls,idx) = generate_batch(batch)
  #print(text.shape)
  #print(offsets.shape)
  print(cls)
  print(idx.cpu().detach().numpy())
  break



tensor([0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0,
        1, 0, 1, 1, 1, 0, 1, 1])
[ 977  962  890  853  809  795  793  774  665  659  417  333  207  139
  130   49   16    3 3403 3373 3050 2592 2420 2177 1757 1581 1398  295
  194 2141 1353  706]


In [47]:
len(train_dataset)

16363

In [48]:
from torch.utils.data import DataLoader

def train_func(train_dataset,test_dataset):

    # Train the model
    train_loss = 0
    train_acc = 0
    train_iterator, valid_iterator = BucketIterator.splits((train_dataset, test_dataset), batch_size = BATCH_SIZE, device = device)

    for i, batch in enumerate(train_iterator):
        (text, offsets, cls,idx) = generate_batch(batch)
        optimizer.zero_grad()
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        output = model(text, offsets)
        loss = criterion(output, cls)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_acc += (output.argmax(1) == cls).sum().item()

    # Adjust the learning rate
    scheduler.step()

    return train_loss / len(train_dataset), train_acc / len(train_dataset)

def test_func(train_dataset,test_dataset):
    loss = 0
    acc = 0
    predicted_idx = []
    predicted_output = []

    train_iterator, valid_iterator = BucketIterator.splits((train_dataset, test_dataset), batch_size = BATCH_SIZE, device = device)

    for i, batch in enumerate(valid_iterator):
        (text, offsets, cls,txt_idx) = generate_batch(batch)
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)

        with torch.no_grad():
            output = model(text, offsets)
            loss = criterion(output, cls)
            loss += loss.item()
            acc += (output.argmax(1) == cls).sum().item()
            #for i in range(output.shape[0]):
            if len(predicted_output) == 0:
              predicted_output = list(output.argmax(1).cpu().detach().numpy())
              predicted_idx = list(txt_idx.cpu().detach().numpy())
            else:
              predicted_output = predicted_output + list(output.argmax(1).cpu().detach().numpy())
              predicted_idx = predicted_idx + list(txt_idx.cpu().detach().numpy())

    return loss / len(test_dataset), acc / len(test_dataset),predicted_output,predicted_idx,output

In [49]:
N_EPOCHS = 20
min_valid_loss = float('inf')

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=4.0)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

for epoch in range(N_EPOCHS):

    start_time = time.time()
    train_loss, train_acc = train_func(train_dataset, test_dataset)
    #valid_loss, valid_acc, valid_idx ,valid_output = test_func(train_dataset,test_dataset)
    valid_loss, valid_acc,valid_output,predicted_idx,output = test_func(train_dataset,test_dataset)

    secs = int(time.time() - start_time)
    mins = secs / 60
    secs = secs % 60

    print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
    print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
    print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)')

Epoch: 1  | time in 0 minutes, 1 seconds
	Loss: 0.0192(train)	|	Acc: 69.2%(train)
	Loss: 0.0006(valid)	|	Acc: 72.9%(valid)
Epoch: 2  | time in 0 minutes, 1 seconds
	Loss: 0.0126(train)	|	Acc: 82.1%(train)
	Loss: 0.0004(valid)	|	Acc: 84.4%(valid)
Epoch: 3  | time in 0 minutes, 1 seconds
	Loss: 0.0104(train)	|	Acc: 85.8%(train)
	Loss: 0.0004(valid)	|	Acc: 85.6%(valid)
Epoch: 4  | time in 0 minutes, 1 seconds
	Loss: 0.0092(train)	|	Acc: 87.7%(train)
	Loss: 0.0004(valid)	|	Acc: 86.4%(valid)
Epoch: 5  | time in 0 minutes, 1 seconds
	Loss: 0.0083(train)	|	Acc: 89.4%(train)
	Loss: 0.0004(valid)	|	Acc: 87.3%(valid)
Epoch: 6  | time in 0 minutes, 1 seconds
	Loss: 0.0077(train)	|	Acc: 90.7%(train)
	Loss: 0.0004(valid)	|	Acc: 87.6%(valid)
Epoch: 7  | time in 0 minutes, 1 seconds
	Loss: 0.0071(train)	|	Acc: 91.6%(train)
	Loss: 0.0004(valid)	|	Acc: 88.0%(valid)
Epoch: 8  | time in 0 minutes, 1 seconds
	Loss: 0.0066(train)	|	Acc: 92.3%(train)
	Loss: 0.0004(valid)	|	Acc: 88.0%(valid)
Epoch: 9  | time

In [50]:
print('Checking the results of test dataset...')
test_loss, test_acc, test_output,test_id, output = test_func(train_dataset, test_dataset)
print(f'\tLoss: {test_loss:.4f}(test)\t|\tAcc: {test_acc * 100:.1f}%(test)')

Checking the results of test dataset...
	Loss: 0.0004(test)	|	Acc: 87.5%(test)


In [51]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

test_results = pd.DataFrame()
test_results["idx"] = test_id
#test_results.set_index("txt_idx",inplace=True)
#test_results["text"] = ""
#test_results["labels"] = -1
#for i in test_idx:
#     idx, label, text = test.__getdata__(i)
#     test_results.loc[i, "text"] = text
#     test_results.loc[i, "labels"] = label

test_results["predicted_output"] = test_output

test_results.head()

,idx,predicted_output
0,977,0
1,962,0
2,890,1
3,853,1
4,809,1


In [52]:
test_data.head()

,textID,text,sentiment,idx,sentiment_label
1,96d74cb729,Shanghai is also really exciting (precisely -- skyscrapers galore). Good tweeps in China: (SH) (BJ).,positive,1,1
2,eee518ae67,"Recession hit Veronique Branquinho, she has to quit her company, such a shame!",negative,2,0
3,01082688c6,happy bday!,positive,3,1
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive,4,1
5,726e501993,that`s great!! weee!! visitors!,positive,5,1


In [53]:
test_results = pd.merge(test_data, test_results, how='left',on="idx")
test_results.head()

,textID,text,sentiment,idx,sentiment_label,predicted_output
0,96d74cb729,Shanghai is also really exciting (precisely -- skyscrapers galore). Good tweeps in China: (SH) (BJ).,positive,1,1,1
1,eee518ae67,"Recession hit Veronique Branquinho, she has to quit her company, such a shame!",negative,2,0,0
2,01082688c6,happy bday!,positive,3,1,1
3,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive,4,1,1
4,726e501993,that`s great!! weee!! visitors!,positive,5,1,1


In [54]:
(test_results[test_results.sentiment_label == test_results.predicted_output].shape[0])/test_results.shape[0]

0.8745247148288974

#### Test the results

In [56]:
from torchtext.data.utils import ngrams_iterator
from torchtext.data.utils import get_tokenizer

ag_news_label = {0 : "Negative",
                 1 : "Positive",
    }

def predict(text, model, vocab, ngrams):
    #tokenizer = get_tokenizer("basic_english")
    with torch.no_grad():
        text = torch.tensor([vocab[token]
                            for token in tokenize_src(text)])
        output = model(text, torch.tensor([0]))
        print (output)
        output = 1 / (1 + np.exp(-output.detach().numpy()))
        print(output.round(3))
        return output.argmax(1).item()

ex_text_str = "MEMPHIS, Tenn. – Four days ago, Jon Rahm was \
    enduring the season’s worst weather conditions on Sunday at The \
    Open on his way to a closing 75 at Royal Portrush, which \
    considering the wind and the rain was a respectable showing. \
    Thursday’s first round at the WGC-FedEx St. Jude Invitational \
    was another story. With temperatures in the mid-80s and hardly any \
    wind, the Spaniard was 13 strokes better in a flawless round. \
    Thanks to his best putting performance on the PGA Tour, Rahm \
    finished with an 8-under 62 for a three-stroke lead, which \
    was even more impressive considering he’d never played the \
    front nine at TPC Southwind."

ex_text_str = "This was a horrible movie!"
#ex_text_str = 'This movie was a ok type movie. I will recommend watching.'

vocab = src_sentences.vocab.stoi
model = model.to("cpu")

print("This is a %s news" %ag_news_label[predict(ex_text_str, model, vocab, 1)])

tensor([[ 9.2047, -9.1373]])
[[1. 0.]]
This is a Negative news


In [57]:
vocab["horrible"]

695

In [58]:
tokenize_src

<function __main__.tokenize_src>

### Save the model embeddings

Saved embeddings will be used to visualize positive and negative words

In [59]:
data_folder

'/content/drive/My Drive/tweet-sentiment-extraction/data'

In [60]:
embedding_csv = model.embedding.weight.cpu().detach().numpy()
np.savetxt(data_folder + "/embedding.csv",embedding_csv,delimiter=',')

In [61]:
embedding_labels = pd.DataFrame(src_sentences.vocab.itos)
embedding_labels.to_csv(data_folder + "/embedding_labels.csv",index=False)